# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [21]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [22]:
world_weather_csv = "worldcity_weather.csv"
world_weather = pd.read_csv(world_weather_csv)
world_weather.head()

,City ID,City,Latitude,Longitude,Max temperature(F),Humidity(%),Cloudiness(%),Wind Speed(mph),Country,Date
0,0,Albany,42.60,-73.97,59.00,94,0,8.97,US,1601521684
1,1,Marzuq,14.40,46.47,72.50,41,0,3.58,YE,1601522201
2,2,Pevek,69.70,170.31,37.09,80,0,7.18,RU,1601521881
3,3,Rawson,-43.30,-65.10,51.58,44,11,24.36,AR,1601522033
4,4,Rikitea,-23.12,-134.97,72.28,82,85,24.70,PF,1601521864


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [23]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [27]:
locations = world_weather[["Latitude", "Longitude"]].astype(float)
humidity = world_weather["Humidity(%)"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=50,
                                 point_radius=2)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [28]:
low_humidity = world_weather.loc[world_weather["Humidity(%)"] < 50, :]
humidityindex = low_humidity.index
ok_weather = world_weather.drop(humidityindex).reset_index(drop=True)

cool_temp = ok_weather.loc[ok_weather["Max temperature(F)"] < 80, :]
tempindex = cool_temp.index
good_weather = ok_weather.drop(tempindex).reset_index(drop=True)

low_wind = good_weather.loc[good_weather["Wind Speed(mph)"] < 10, :]
windindex = low_wind.index
great_weather = good_weather.drop(windindex).reset_index(drop=True)

low_clouds = great_weather.loc[great_weather["Cloudiness(%)"] < 25, :]
cloudindex = low_clouds.index
fantastic_weather = great_weather.drop(cloudindex).reset_index(drop=True)

fantastic_weather.count()

City ID               15
City                  15
Latitude              15
Longitude             15
Max temperature(F)    15
Humidity(%)           15
Cloudiness(%)         15
Wind Speed(mph)       15
Country               15
Date                  15
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [29]:
hotel_df = pd.DataFrame(fantastic_weather, columns=["City", "Country", "Latitude", "Longitude"])
hotel_df['Hotel Name'] = ""
hotel_df.head()

,City,Country,Latitude,Longitude,Hotel Name
0,Mataram,ID,-8.58,116.12,
1,Port Blair,IN,11.67,92.75,
2,Kavaratti,IN,10.57,72.64,
3,Gushikawa,JP,26.35,127.87,
4,Itoman,JP,26.12,127.67,


In [35]:
hotel_list = []

for i in range(len(hotel_df)):
    lat = hotel_df.loc[i]['Latitude']
    lon = hotel_df.loc[i]['Longitude']
    
    params = {
        "location": f"{lat},{lon}",
        "radius": 5000,
        "types": "lodging",
        "key": g_key
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params)
    results = response.json()
    try:
        hotel_list.append(results['results'][0]['name'])
    except:
        hotel_list.append("")
hotel_df["Hotel Name"] = hotel_list
hotel_df = hotel_df.dropna(how='all')
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
0,Mataram,ID,-8.58,116.12,Hotel Santika Mataram
1,Port Blair,IN,11.67,92.75,Fortune Resort Bay Island
2,Kavaratti,IN,10.57,72.64,Hotels in Lakshadweep Islands
3,Gushikawa,JP,26.35,127.87,Kirari Guest House
4,Itoman,JP,26.12,127.67,Southern Beach Hotel & Resort Okinawa
5,Tual,ID,-5.67,132.75,Grand Vilia Hotel
6,Bathsheba,BB,13.22,-59.52,Atlantis Hotel
7,Ugoofaaru,MV,5.67,73.00,Ungoofaaru Inn
8,Manadhoo,MV,5.77,73.38,Vevu Guesthouse
9,Merauke,ID,-8.47,140.33,


In [31]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [36]:
# Add marker layer ontop of heat map
marker_locations = hotel_df[["Latitude", "Longitude"]]
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))